<a href="https://colab.research.google.com/github/sultanardian/Deep-Learning-and-Data-Science/blob/main/Sigarar_Utang_Coffee_Variety_Identification_using_SVM_and_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Library

In [ ]:
from google.colab.patches import cv2_imshow

In [ ]:
# import library
import os
import numpy as np
import random
from tqdm import tqdm
import pandas as pd
from random import seed
from random import randint

import cv2
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import ConfusionMatrixDisplay, classification_report, confusion_matrix, accuracy_score

from skimage.feature import greycomatrix, greycoprops
from skimage.io import imread
import skimage

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

# import keras_tuner as kt

# Preparing datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
PATH = '/content/drive/MyDrive/Job/projects.co.id/dataset/Dataset'
CATS = ['cacat', 'tidak_cacat']

In [ ]:
asset = []

for cat in CATS:
    img_path = os.path.join(PATH, cat, 'original')
    for files in tqdm(sorted(os.listdir(img_path))):
        dir = os.path.join(img_path, files)
        img = cv2.imread(dir)
        asset.append({
            'img' : img,
            'label' : cat,
            'filename' : files
        })

100%|██████████| 160/160 [00:22<00:00,  7.23it/s]


In [ ]:
len(asset)

320

# Define some functions

### Preprocessing functions

In [ ]:
def rotate_augmentation(data, angles = []):
    img = data['img']
    label = data['label']
    filename = data['filename'].split('.')[0]

    new_asset = []
    angles = angles

    if angles.count(0) == 0:
        angles.append(0)

    for angle in angles:

        (h, w) = img.shape[:2]
        center = (w / 2, h / 2)
        scale = 1

        M = cv2.getRotationMatrix2D(center, angle, scale)
        result = cv2.warpAffine(img, M, (w, h), borderValue=(255,255,255))

        file_name = '{}-{}.png'.format(filename, angle)
        file_loc = os.path.join(PATH, label, 'augmentation')
        save_img(result, file_loc, file_name)

        new_asset.append({
            'img' : result,
            'label' : label,
            'filename' : file_name
        })
       
    return new_asset

def cropping(data):
    img = data['img']
    label = data['label']
    filename = data['filename'].split('.')[0]

    if round(img.shape[0]/img.shape[1],1) == 0.8:
        y = int((img.shape[0]/2)-(img.shape[0]/5))
        x = int((img.shape[1]/2)-((img.shape[1]/40)*3))
        h = int(img.shape[0]/3)
        w = int(img.shape[0]/3)
        result = img[y:y+h,x:x+w]
    elif round(img.shape[0]/img.shape[1],1) == 2.2:
        y = int((img.shape[0]/2)-(img.shape[0]/5))
        x = int((img.shape[1]/2)-((img.shape[1]/8)*4))
        h = int(img.shape[0]/2)
        w = int(img.shape[0]/2)
        result = img[y:y+h,x:x+w]
    else:
        y = int((img.shape[0]/2)-(img.shape[0]/4))
        x = int((img.shape[1]/2)-((img.shape[1]/30)*3))
        h = int(img.shape[0]/2)
        w = int(img.shape[0]/2)
        result = img[y:y+h,x:x+w]

    file_name = '{}.png'.format(filename)
    file_loc = os.path.join(PATH, label, 'cropping')
    save_img(result, file_loc, file_name)

    cropped_img = {
        'img' : result,
        'label' : label,
        'filename' : file_name
    }

    return cropped_img

def resize(data, height, width):
    img = data['img']
    label = data['label']
    filename = data['filename'].split('.')[0]

    result = cv2.resize(img, (height, width))

    file_name = '{}.png'.format(filename)
    file_loc = os.path.join(PATH, label, 'resize')
    save_img(result, file_loc, file_name)

    resize_img = {
        'img' : result,
        'label' : label,
        'filename' : file_name
    }

    return resize_img

def extract_foreground(data):
    img = data['img']
    label = data['label']
    filename = data['filename'].split('.')[0]

    # convert BGR to HSV
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # convert BGR to YUV for check image contrast
    Y = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)[:,:,0]

    # compute min and max of Y
    min_Y = np.min(Y)
    max_Y = np.max(Y)

    # compute contrast
    contrast = (max_Y-min_Y)/(max_Y+min_Y)

    if contrast < 1.0:
        hsv_lower = np.array([0,0,150])
        hsv_upper = np.array([179,255,255])
        X = 0
        y = 255
    elif 1.0 <= contrast < 10.0:
        hsv_lower = np.array([0,0,215])
        hsv_upper = np.array([179,255,255])
        X = 125
        y = 255
    elif 10.0 < contrast < 12.0:
        hsv_lower = np.array([0,0,155])
        hsv_upper = np.array([179,255,255])
        X = 185
        y = 255
    else:
        hsv_lower = np.array([0,0,155])
        hsv_upper = np.array([179,255,255])
        X = 0
        y = 255

    mask = cv2.inRange(hsv, hsv_lower, hsv_upper)
    result = cv2.bitwise_and(img, img, mask=mask)

    result_gray = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)
    
    (thresh, result_bnw) = cv2.threshold(result_gray, X, y, cv2.THRESH_BINARY)

    file_name = '{}.png'.format(filename)
    file_loc = os.path.join(PATH, label, 'foreground')
    save_img(result_bnw, file_loc, file_name)

    foreground_img = {
        'img' : result_bnw,
        'label' : label,
        'filename' : file_name
    }
    
    return foreground_img

### Feature extraction functions

In [ ]:
def geometry_features(img):
        img = cv2.GaussianBlur(img,(5,5),0)
        img = cv2.Canny(img, 10, 150)

        contours, _ = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        # get diameters
        diameters = []
        for i in contours:
            for j in i:
                diameters.append(j[0][1] - j[0][0])
        diameter = max([abs(d) for d in diameters])
        
        cnt = contours[0]

        area = cv2.contourArea(cnt)
        perimeter = cv2.arcLength(cnt,True)
        x,y,w,l = cv2.boundingRect(cnt)
        return {
            'area' : round(area, 2),
            'perimeter' : round(perimeter, 2),
            'width' : round(w, 2),
            'length' : round(l, 2),
            'diameter' : round(diameter, 2)
        }


def morphology_features(img):
    geom_feats = geometry_features(img)
    
    return {
        'Ar' : round(geom_feats['length']/geom_feats['width'], 2),
        'F' : 0 if geom_feats['area'] == 0 else round(4*3.14*geom_feats['area']/geom_feats['perimeter']**2, 2),
        'Rect' : 0 if geom_feats['area'] == 0 else round(geom_feats['length']*geom_feats['width']/geom_feats['area'], 2),
        'N' : round(geom_feats['diameter']/geom_feats['length'], 2),
        'Pd' : round(geom_feats['perimeter']/geom_feats['diameter'], 2),
        'Pplf' : round(geom_feats['perimeter']/(geom_feats['length']+geom_feats['width']), 2)
    }

### Additional function

In [ ]:
def save_img(img, dir, filename):
    while True:
        if os.path.isdir(dir):
            path = os.path.join(dir, filename)
            cv2.imwrite(path, img)
            break
        else:
            os.mkdir(dir)

# Image preprocessing

In [ ]:
crop_img = []
for data in asset:
    result = cropping(data)
    crop_img.append(result)


res_img = []
for data in crop_img:
    result = resize(data, 256, 256)
    res_img.append(result)


aug_img = []
for data in res_img:
    results = rotate_augmentation(data, [30, 40, 60, 90])
    for result in results:
        aug_img.append(result)


fore_img = []
for data in aug_img:
    result = extract_foreground(data)
    fore_img.append(result)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:104: RuntimeWarning: overflow encountered in ubyte_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:104: RuntimeWarning: divide by zero encountered in ubyte_scalars


# Feature extraction

In [ ]:
labels = [data['label'] for data in fore_img]

In [ ]:
geoms = pd.DataFrame([geometry_features(data['img']) for data in fore_img])
morphs = pd.DataFrame([morphology_features(data['img']) for data in fore_img])
labels = pd.DataFrame(labels, columns = ['Label'])

# Define dataframe

In [ ]:
df_raw = pd.concat([geoms, morphs, labels], axis=1, join='inner')

In [ ]:
df_raw

,area,perimeter,width,length,diameter,Ar,F,Rect,N,Pd,Pplf,Label
0,0.0,0.00,1,1,124,1.00,0.00,0.00,124.00,0.00,0.00,cacat
1,0.0,6.00,4,1,130,0.25,0.00,0.00,130.00,0.05,1.20,cacat
2,1.5,6.24,3,3,135,1.00,0.48,6.00,45.00,0.05,1.04,cacat
3,0.5,6.24,3,2,141,0.67,0.16,12.00,70.50,0.04,1.25,cacat
4,6.0,10.00,3,4,94,1.33,0.75,2.00,23.50,0.11,1.43,cacat
...,...,...,...,...,...,...,...,...,...,...,...,...
1595,8381.0,354.05,98,115,100,1.17,0.84,1.34,0.87,3.54,1.66,tidak_cacat
1596,3.0,18.49,5,4,171,0.80,0.11,6.67,42.75,0.11,2.05,tidak_cacat
1597,8376.5,352.98,88,123,84,1.40,0.84,1.29,0.68,4.20,1.67,tidak_cacat
1598,8387.5,351.40,101,118,89,1.17,0.85,1.42,0.75,3.95,1.60,tidak_cacat


In [ ]:
df_raw.to_csv(os.path.join(PATH, 'dataset.csv'), index = False)

In [ ]:
df_raw = pd.read_csv(os.path.join(PATH, 'dataset.csv'))

# Modelling

In [ ]:
df = df_raw.copy()

### Splitting dataset

In [ ]:
X = df.drop('Label', axis = 1)
y = df['Label']

#### 10:90

In [ ]:
X_train_10_90, X_test_10_90, y_train_10_90, y_test_10_90 = train_test_split(X, y, test_size = 0.9, random_state = 42, stratify = y)

#### 20:80

In [ ]:
X_train_20_80, X_test_20_80, y_train_20_80, y_test_20_80 = train_test_split(X, y, test_size = 0.8, random_state = 42, stratify = y)

#### 30:70

In [ ]:
X_train_30_70, X_test_30_70, y_train_30_70, y_test_30_70 = train_test_split(X, y, test_size = 0.7, random_state = 42, stratify = y)

#### 40:60

In [ ]:
X_train_40_60, X_test_40_60, y_train_40_60, y_test_40_60 = train_test_split(X, y, test_size = 0.6, random_state = 42, stratify = y)

#### 50:50

In [ ]:
X_train_50_50, X_test_50_50, y_train_50_50, y_test_50_50 = train_test_split(X, y, test_size = 0.5, random_state = 42, stratify = y)

#### 60:40

In [ ]:
X_train_60_40, X_test_60_40, y_train_60_40, y_test_60_40 = train_test_split(X, y, test_size = 0.4, random_state = 42, stratify = y)

#### 70:30

In [ ]:
X_train_70_30, X_test_70_30, y_train_70_30, y_test_70_30 = train_test_split(X, y, test_size = 0.3, random_state = 42, stratify = y)

#### 80:20

In [ ]:
X_train_80_20, X_test_80_20, y_train_80_20, y_test_80_20 = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = y)

#### 90:10

In [ ]:
X_train_90_10, X_test_90_10, y_train_90_10, y_test_90_10 = train_test_split(X, y, test_size = 0.1, random_state = 42, stratify = y)

### Training (SVM) models

#### 10:90

##### Kernel RBF

In [ ]:
SVM_10_90_RBF = SVC(kernel = 'rbf')
SVM_10_90_RBF.fit(X_train_10_90, y_train_10_90)

SVC()

##### Kernel Polynomial

In [ ]:
SVM_10_90_poly = SVC(kernel = 'poly')
SVM_10_90_poly.fit(X_train_10_90, y_train_10_90)

SVC(kernel='poly')

##### Kernel Sigmoid

In [ ]:
SVM_10_90_sigmoid = SVC(kernel = 'sigmoid')
SVM_10_90_sigmoid.fit(X_train_10_90, y_train_10_90)

SVC(kernel='sigmoid')

#### 20:80

##### Kernel RBF

In [ ]:
SVM_20_80_RBF = SVC(kernel = 'rbf')
SVM_20_80_RBF.fit(X_train_20_80, y_train_20_80)

SVC()

##### Kernel Polynomial

In [ ]:
SVM_20_80_poly = SVC(kernel = 'poly')
SVM_20_80_poly.fit(X_train_20_80, y_train_20_80)

SVC(kernel='poly')

##### Kernel Sigmoid

In [ ]:
SVM_20_80_sigmoid = SVC(kernel = 'sigmoid')
SVM_20_80_sigmoid.fit(X_train_20_80, y_train_20_80)

SVC(kernel='sigmoid')

#### 30:70

##### Kernel RBF

In [ ]:
SVM_30_70_RBF = SVC(kernel = 'rbf')
SVM_30_70_RBF.fit(X_train_30_70, y_train_30_70)

SVC()

##### Kernel Polynomial

In [ ]:
SVM_30_70_poly = SVC(kernel = 'poly')
SVM_30_70_poly.fit(X_train_30_70, y_train_30_70)

SVC(kernel='poly')

##### Kernel Sigmoid

In [ ]:
SVM_30_70_sigmoid = SVC(kernel = 'sigmoid')
SVM_30_70_sigmoid.fit(X_train_30_70, y_train_30_70)

SVC(kernel='sigmoid')

#### 40:60

##### Kernel RBF

In [ ]:
SVM_40_60_RBF = SVC(kernel = 'rbf')
SVM_40_60_RBF.fit(X_train_40_60, y_train_40_60)

SVC()

##### Kernel Polynomial

In [ ]:
SVM_40_60_poly = SVC(kernel = 'poly')
SVM_40_60_poly.fit(X_train_40_60, y_train_40_60)

SVC(kernel='poly')

##### Kernel Sigmoid

In [ ]:
SVM_40_60_sigmoid = SVC(kernel = 'sigmoid')
SVM_40_60_sigmoid.fit(X_train_40_60, y_train_40_60)

SVC(kernel='sigmoid')

#### 50:50

##### Kernel RBF

In [ ]:
SVM_50_50_RBF = SVC(kernel = 'rbf')
SVM_50_50_RBF.fit(X_train_50_50, y_train_50_50)

SVC()

##### Kernel Polynomial

In [ ]:
SVM_50_50_poly = SVC(kernel = 'poly')
SVM_50_50_poly.fit(X_train_50_50, y_train_50_50)

SVC(kernel='poly')

##### Kernel Sigmoid

In [ ]:
SVM_50_50_sigmoid = SVC(kernel = 'sigmoid')
SVM_50_50_sigmoid.fit(X_train_50_50, y_train_50_50)

SVC(kernel='sigmoid')

#### 60:40

##### Kernel RBF

In [ ]:
SVM_60_40_RBF = SVC(kernel = 'rbf')
SVM_60_40_RBF.fit(X_train_60_40, y_train_60_40)

SVC()

##### Kernel Polynomial

In [ ]:
SVM_60_40_poly = SVC(kernel = 'poly')
SVM_60_40_poly.fit(X_train_60_40, y_train_60_40)

SVC(kernel='poly')

##### Kernel Sigmoid

In [ ]:
SVM_60_40_sigmoid = SVC(kernel = 'sigmoid')
SVM_60_40_sigmoid.fit(X_train_60_40, y_train_60_40)

SVC(kernel='sigmoid')

#### 70:30

##### Kernel RBF

In [ ]:
SVM_70_30_RBF = SVC(kernel = 'rbf')
SVM_70_30_RBF.fit(X_train_70_30, y_train_70_30)

SVC()

##### Kernel Polynomial

In [ ]:
SVM_70_30_poly = SVC(kernel = 'poly')
SVM_70_30_poly.fit(X_train_70_30, y_train_70_30)

SVC(kernel='poly')

##### Kernel Sigmoid

In [ ]:
SVM_70_30_sigmoid = SVC(kernel = 'sigmoid')
SVM_70_30_sigmoid.fit(X_train_70_30, y_train_70_30)

SVC(kernel='sigmoid')

#### 80:20

##### Kernel RBF

In [ ]:
SVM_80_20_RBF = SVC(kernel = 'rbf')
SVM_80_20_RBF.fit(X_train_80_20, y_train_80_20)

SVC()

##### Kernel Polynomial

In [ ]:
SVM_80_20_poly = SVC(kernel = 'poly')
SVM_80_20_poly.fit(X_train_80_20, y_train_80_20)

SVC(kernel='poly')

##### Kernel Sigmoid

In [ ]:
SVM_80_20_sigmoid = SVC(kernel = 'sigmoid')
SVM_80_20_sigmoid.fit(X_train_80_20, y_train_80_20)

SVC(kernel='sigmoid')

#### 90:10

##### Kernel RBF

In [ ]:
SVM_90_10_RBF = SVC(kernel = 'rbf')
SVM_90_10_RBF.fit(X_train_90_10, y_train_90_10)

SVC()

##### Kernel Polynomial

In [ ]:
SVM_90_10_poly = SVC(kernel = 'poly')
SVM_90_10_poly.fit(X_train_90_10, y_train_90_10)

SVC(kernel='poly')

##### Kernel Sigmoid

In [ ]:
SVM_90_10_sigmoid = SVC(kernel = 'sigmoid')
SVM_90_10_sigmoid.fit(X_train_90_10, y_train_90_10)

SVC(kernel='sigmoid')

### Training (ANN) models

#### 10:90

In [ ]:
X_train_10_90_np = np.array(X_train_10_90)
X_test_10_90_np = np.array(X_test_10_90)

LE = LabelEncoder()
y_train_10_90_enc = LE.fit_transform(y_train_10_90)
y_test_10_90_enc = LE.fit_transform(y_test_10_90)

In [ ]:
ANN_10_90 = Sequential()

ANN_10_90.add(Dense(units = 16, activation = 'relu', input_shape = (df.shape[1]-1,)))
ANN_10_90.add(Dense(units = 16, activation = 'relu'))
ANN_10_90.add(Dense(units = 1, activation = 'sigmoid'))
ANN_10_90.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ['accuracy'])

ANN_10_90.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_25 (Dense)            (None, 16)                192       
                                                                 
 dense_26 (Dense)            (None, 16)                272       
                                                                 
 dense_27 (Dense)            (None, 1)                 17        
                                                                 
Total params: 481
Trainable params: 481
Non-trainable params: 0
_________________________________________________________________


In [ ]:
ANN_10_90.fit(X_train_10_90_np, y_train_10_90_enc, epochs = 100, validation_split = 0.2)

Epoch 1/100
4/4 [==============================] - 1s 98ms/step - loss: 144.5197 - accuracy: 0.5312 - val_loss: 117.6429 - val_accuracy: 0.4688
Epoch 2/100
4/4 [==============================] - 0s 14ms/step - loss: 120.4804 - accuracy: 0.5469 - val_loss: 96.3179 - val_accuracy: 0.5000
Epoch 3/100
4/4 [==============================] - 0s 17ms/step - loss: 97.9818 - accuracy: 0.5547 - val_loss: 75.7249 - val_accuracy: 0.6250
Epoch 4/100
4/4 [==============================] - 0s 15ms/step - loss: 74.9416 - accuracy: 0.6016 - val_loss: 56.1033 - val_accuracy: 0.5938
Epoch 5/100
4/4 [==============================] - 0s 14ms/step - loss: 52.7180 - accuracy: 0.6328 - val_loss: 36.8701 - val_accuracy: 0.6250
Epoch 6/100
4/4 [==============================] - 0s 13ms/step - loss: 30.4875 - accuracy: 0.6641 - val_loss: 17.9480 - val_accuracy: 0.6250
Epoch 7/100
4/4 [==============================] - 0s 15ms/step - loss: 12.1538 - accuracy: 0.5781 - val_loss: 7.6382 - val_accuracy: 0.5625
Epoc

#### 20:80

In [ ]:
X_train_20_80_np = np.array(X_train_20_80)
X_test_20_80_np = np.array(X_test_20_80)

LE = LabelEncoder()
y_train_20_80_enc = LE.fit_transform(y_train_20_80)
y_test_20_80_enc = LE.fit_transform(y_test_20_80)

In [ ]:
ANN_20_80 = Sequential()

ANN_20_80.add(Dense(units = 16, activation = 'relu', input_shape = (df.shape[1]-1,)))
ANN_20_80.add(Dense(units = 16, activation = 'relu'))
ANN_20_80.add(Dense(units = 1, activation = 'sigmoid'))
ANN_20_80.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ['accuracy'])

ANN_20_80.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_22 (Dense)            (None, 16)                192       
                                                                 
 dense_23 (Dense)            (None, 16)                272       
                                                                 
 dense_24 (Dense)            (None, 1)                 17        
                                                                 
Total params: 481
Trainable params: 481
Non-trainable params: 0
_________________________________________________________________


In [ ]:
ANN_20_80.fit(X_train_20_80_np, y_train_20_80_enc, epochs = 100, validation_split = 0.2)

Epoch 1/100
8/8 [==============================] - 0s 15ms/step - loss: 0.6484 - accuracy: 0.9023 - val_loss: 0.3804 - val_accuracy: 0.8906
Epoch 2/100
8/8 [==============================] - 0s 10ms/step - loss: 0.6074 - accuracy: 0.9258 - val_loss: 0.3873 - val_accuracy: 0.8906
Epoch 3/100
8/8 [==============================] - 0s 11ms/step - loss: 0.5937 - accuracy: 0.9180 - val_loss: 0.3414 - val_accuracy: 0.8906
Epoch 4/100
8/8 [==============================] - 0s 12ms/step - loss: 0.5494 - accuracy: 0.9219 - val_loss: 0.3535 - val_accuracy: 0.8906
Epoch 5/100
8/8 [==============================] - 0s 10ms/step - loss: 0.5294 - accuracy: 0.9219 - val_loss: 0.3488 - val_accuracy: 0.8906
Epoch 6/100
8/8 [==============================] - 0s 17ms/step - loss: 0.4978 - accuracy: 0.9336 - val_loss: 0.3371 - val_accuracy: 0.8906
Epoch 7/100
8/8 [==============================] - 0s 12ms/step - loss: 0.4650 - accuracy: 0.9180 - val_loss: 0.3414 - val_accuracy: 0.8906
Epoch 8/100
8/8 [===

#### 30:70

In [ ]:
X_train_30_70_np = np.array(X_train_30_70)
X_test_30_70_np = np.array(X_test_30_70)

LE = LabelEncoder()
y_train_30_70_enc = LE.fit_transform(y_train_30_70)
y_test_30_70_enc = LE.fit_transform(y_test_30_70)

In [ ]:
ANN_30_70 = Sequential()

ANN_30_70.add(Dense(units = 16, activation = 'relu', input_shape = (df.shape[1]-1,)))
ANN_30_70.add(Dense(units = 16, activation = 'relu'))
ANN_30_70.add(Dense(units = 1, activation = 'sigmoid'))
ANN_30_70.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ['accuracy'])

ANN_30_70.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_28 (Dense)            (None, 16)                192       
                                                                 
 dense_29 (Dense)            (None, 16)                272       
                                                                 
 dense_30 (Dense)            (None, 1)                 17        
                                                                 
Total params: 481
Trainable params: 481
Non-trainable params: 0
_________________________________________________________________


In [ ]:
ANN_30_70.fit(X_train_30_70_np, y_train_30_70_enc, epochs = 100, validation_split = 0.2)

Epoch 1/100
12/12 [==============================] - 1s 30ms/step - loss: 32.4008 - accuracy: 0.6927 - val_loss: 21.9372 - val_accuracy: 0.6354
Epoch 2/100
12/12 [==============================] - 0s 7ms/step - loss: 13.8747 - accuracy: 0.6979 - val_loss: 5.9605 - val_accuracy: 0.6562
Epoch 3/100
12/12 [==============================] - 0s 7ms/step - loss: 3.6959 - accuracy: 0.6927 - val_loss: 2.7356 - val_accuracy: 0.4479
Epoch 4/100
12/12 [==============================] - 0s 7ms/step - loss: 2.1186 - accuracy: 0.6562 - val_loss: 2.0905 - val_accuracy: 0.5000
Epoch 5/100
12/12 [==============================] - 0s 8ms/step - loss: 1.5275 - accuracy: 0.6901 - val_loss: 2.2440 - val_accuracy: 0.4896
Epoch 6/100
12/12 [==============================] - 0s 7ms/step - loss: 1.2816 - accuracy: 0.6328 - val_loss: 1.8434 - val_accuracy: 0.4479
Epoch 7/100
12/12 [==============================] - 0s 7ms/step - loss: 1.1839 - accuracy: 0.6458 - val_loss: 1.8079 - val_accuracy: 0.4688
Epoch 8/1

#### 40:60

In [ ]:
X_train_40_60_np = np.array(X_train_40_60)
X_test_40_60_np = np.array(X_test_40_60)

LE = LabelEncoder()
y_train_40_60_enc = LE.fit_transform(y_train_40_60)
y_test_40_60_enc = LE.fit_transform(y_test_40_60)

In [ ]:
ANN_40_60 = Sequential()

ANN_40_60.add(Dense(units = 16, activation = 'relu', input_shape = (df.shape[1]-1,)))
ANN_40_60.add(Dense(units = 16, activation = 'relu'))
ANN_40_60.add(Dense(units = 1, activation = 'sigmoid'))
ANN_40_60.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ['accuracy'])

ANN_40_60.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_31 (Dense)            (None, 16)                192       
                                                                 
 dense_32 (Dense)            (None, 16)                272       
                                                                 
 dense_33 (Dense)            (None, 1)                 17        
                                                                 
Total params: 481
Trainable params: 481
Non-trainable params: 0
_________________________________________________________________


In [ ]:
ANN_40_60.fit(X_train_40_60_np, y_train_40_60_enc, epochs = 100, validation_split = 0.2)

Epoch 1/100
16/16 [==============================] - 1s 24ms/step - loss: 70.2185 - accuracy: 0.5078 - val_loss: 52.2191 - val_accuracy: 0.4688
Epoch 2/100
16/16 [==============================] - 0s 6ms/step - loss: 25.5072 - accuracy: 0.5098 - val_loss: 4.4554 - val_accuracy: 0.5234
Epoch 3/100
16/16 [==============================] - 0s 6ms/step - loss: 4.4976 - accuracy: 0.5547 - val_loss: 3.6807 - val_accuracy: 0.5547
Epoch 4/100
16/16 [==============================] - 0s 6ms/step - loss: 1.5345 - accuracy: 0.7324 - val_loss: 1.0509 - val_accuracy: 0.7500
Epoch 5/100
16/16 [==============================] - 0s 6ms/step - loss: 0.6691 - accuracy: 0.8223 - val_loss: 0.7774 - val_accuracy: 0.8125
Epoch 6/100
16/16 [==============================] - 0s 6ms/step - loss: 0.4420 - accuracy: 0.8848 - val_loss: 0.6409 - val_accuracy: 0.8750
Epoch 7/100
16/16 [==============================] - 0s 6ms/step - loss: 0.3671 - accuracy: 0.8926 - val_loss: 0.5768 - val_accuracy: 0.8750
Epoch 8/1

#### 50:50

In [ ]:
X_train_50_50_np = np.array(X_train_50_50)
X_test_50_50_np = np.array(X_test_50_50)

LE = LabelEncoder()
y_train_50_50_enc = LE.fit_transform(y_train_50_50)
y_test_50_50_enc = LE.fit_transform(y_test_50_50)

In [ ]:
ANN_50_50 = Sequential()

ANN_50_50.add(Dense(units = 16, activation = 'relu', input_shape = (df.shape[1]-1,)))
ANN_50_50.add(Dense(units = 16, activation = 'relu'))
ANN_50_50.add(Dense(units = 1, activation = 'sigmoid'))
ANN_50_50.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ['accuracy'])

ANN_50_50.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_34 (Dense)            (None, 16)                192       
                                                                 
 dense_35 (Dense)            (None, 16)                272       
                                                                 
 dense_36 (Dense)            (None, 1)                 17        
                                                                 
Total params: 481
Trainable params: 481
Non-trainable params: 0
_________________________________________________________________


In [ ]:
ANN_50_50.fit(X_train_50_50_np, y_train_50_50_enc, epochs = 100, validation_split = 0.2)

Epoch 1/100
20/20 [==============================] - 2s 30ms/step - loss: 154.4009 - accuracy: 0.4781 - val_loss: 31.6324 - val_accuracy: 0.4313
Epoch 2/100
20/20 [==============================] - 0s 8ms/step - loss: 12.7680 - accuracy: 0.6016 - val_loss: 12.9498 - val_accuracy: 0.6812
Epoch 3/100
20/20 [==============================] - 0s 7ms/step - loss: 5.8053 - accuracy: 0.6672 - val_loss: 0.7171 - val_accuracy: 0.7437
Epoch 4/100
20/20 [==============================] - 0s 7ms/step - loss: 0.7305 - accuracy: 0.7875 - val_loss: 0.2926 - val_accuracy: 0.9062
Epoch 5/100
20/20 [==============================] - 0s 5ms/step - loss: 0.2801 - accuracy: 0.9016 - val_loss: 0.2385 - val_accuracy: 0.9125
Epoch 6/100
20/20 [==============================] - 0s 5ms/step - loss: 0.2372 - accuracy: 0.9219 - val_loss: 0.2141 - val_accuracy: 0.9438
Epoch 7/100
20/20 [==============================] - 0s 6ms/step - loss: 0.2317 - accuracy: 0.9156 - val_loss: 0.2151 - val_accuracy: 0.9375
Epoch 8

#### 60:40

In [ ]:
X_train_60_40_np = np.array(X_train_60_40)
X_test_60_40_np = np.array(X_test_60_40)

LE = LabelEncoder()
y_train_60_40_enc = LE.fit_transform(y_train_60_40)
y_test_60_40_enc = LE.fit_transform(y_test_60_40)

In [ ]:
ANN_60_40 = Sequential()

ANN_60_40.add(Dense(units = 16, activation = 'relu', input_shape = (df.shape[1]-1,)))
ANN_60_40.add(Dense(units = 16, activation = 'relu'))
ANN_60_40.add(Dense(units = 1, activation = 'sigmoid'))
ANN_60_40.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ['accuracy'])

ANN_60_40.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_40 (Dense)            (None, 16)                192       
                                                                 
 dense_41 (Dense)            (None, 16)                272       
                                                                 
 dense_42 (Dense)            (None, 1)                 17        
                                                                 
Total params: 481
Trainable params: 481
Non-trainable params: 0
_________________________________________________________________


In [ ]:
ANN_60_40.fit(X_train_60_40_np, y_train_60_40_enc, epochs = 100, validation_split = 0.2)

Epoch 1/100
24/24 [==============================] - 1s 31ms/step - loss: 10.8587 - accuracy: 0.5729 - val_loss: 3.8074 - val_accuracy: 0.6875
Epoch 2/100
24/24 [==============================] - 0s 5ms/step - loss: 1.6289 - accuracy: 0.6589 - val_loss: 1.1419 - val_accuracy: 0.6927
Epoch 3/100
24/24 [==============================] - 0s 5ms/step - loss: 0.9873 - accuracy: 0.6654 - val_loss: 0.8081 - val_accuracy: 0.6406
Epoch 4/100
24/24 [==============================] - 0s 5ms/step - loss: 0.7683 - accuracy: 0.7031 - val_loss: 0.6912 - val_accuracy: 0.6562
Epoch 5/100
24/24 [==============================] - 0s 5ms/step - loss: 0.6260 - accuracy: 0.6940 - val_loss: 0.5875 - val_accuracy: 0.6667
Epoch 6/100
24/24 [==============================] - 0s 5ms/step - loss: 0.7537 - accuracy: 0.7044 - val_loss: 0.5675 - val_accuracy: 0.7135
Epoch 7/100
24/24 [==============================] - 0s 5ms/step - loss: 0.5271 - accuracy: 0.7461 - val_loss: 0.5007 - val_accuracy: 0.7552
Epoch 8/100

#### 70:30

In [ ]:
X_train_70_30_np = np.array(X_train_70_30)
X_test_70_30_np = np.array(X_test_70_30)

LE = LabelEncoder()
y_train_70_30_enc = LE.fit_transform(y_train_70_30)
y_test_70_30_enc = LE.fit_transform(y_test_70_30)

In [ ]:
ANN_70_30 = Sequential()

ANN_70_30.add(Dense(units = 16, activation = 'relu', input_shape = (df.shape[1]-1,)))
ANN_70_30.add(Dense(units = 16, activation = 'relu'))
ANN_70_30.add(Dense(units = 1, activation = 'sigmoid'))
ANN_70_30.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ['accuracy'])

ANN_70_30.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_43 (Dense)            (None, 16)                192       
                                                                 
 dense_44 (Dense)            (None, 16)                272       
                                                                 
 dense_45 (Dense)            (None, 1)                 17        
                                                                 
Total params: 481
Trainable params: 481
Non-trainable params: 0
_________________________________________________________________


In [ ]:
ANN_70_30.fit(X_train_70_30_np, y_train_70_30_enc, epochs = 100, validation_split = 0.2)

Epoch 1/100
28/28 [==============================] - 2s 26ms/step - loss: 132.5890 - accuracy: 0.4777 - val_loss: 32.1112 - val_accuracy: 0.4018
Epoch 2/100
28/28 [==============================] - 0s 7ms/step - loss: 10.0433 - accuracy: 0.6484 - val_loss: 8.5119 - val_accuracy: 0.7545
Epoch 3/100
28/28 [==============================] - 0s 12ms/step - loss: 5.2561 - accuracy: 0.6395 - val_loss: 2.4543 - val_accuracy: 0.7500
Epoch 4/100
28/28 [==============================] - 0s 5ms/step - loss: 1.8755 - accuracy: 0.6384 - val_loss: 1.3785 - val_accuracy: 0.7455
Epoch 5/100
28/28 [==============================] - 0s 10ms/step - loss: 1.3612 - accuracy: 0.6607 - val_loss: 1.1331 - val_accuracy: 0.6562
Epoch 6/100
28/28 [==============================] - 0s 5ms/step - loss: 1.1115 - accuracy: 0.6518 - val_loss: 0.9160 - val_accuracy: 0.7366
Epoch 7/100
28/28 [==============================] - 0s 11ms/step - loss: 0.8794 - accuracy: 0.6819 - val_loss: 0.7243 - val_accuracy: 0.7545
Epoch

#### 80:20

In [ ]:
X_train_80_20_np = np.array(X_train_80_20)
X_test_80_20_np = np.array(X_test_80_20)

LE = LabelEncoder()
y_train_80_20_enc = LE.fit_transform(y_train_80_20)
y_test_80_20_enc = LE.fit_transform(y_test_80_20)

In [ ]:
ANN_80_20 = Sequential()

ANN_80_20.add(Dense(units = 16, activation = 'relu', input_shape = (df.shape[1]-1,)))
ANN_80_20.add(Dense(units = 16, activation = 'relu'))
ANN_80_20.add(Dense(units = 1, activation = 'sigmoid'))
ANN_80_20.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ['accuracy'])

ANN_80_20.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_46 (Dense)            (None, 16)                192       
                                                                 
 dense_47 (Dense)            (None, 16)                272       
                                                                 
 dense_48 (Dense)            (None, 1)                 17        
                                                                 
Total params: 481
Trainable params: 481
Non-trainable params: 0
_________________________________________________________________


In [ ]:
ANN_80_20.fit(X_train_80_20_np, y_train_80_20_enc, epochs = 100, validation_split = 0.2)

Epoch 1/100
32/32 [==============================] - 1s 12ms/step - loss: 20.1886 - accuracy: 0.6621 - val_loss: 6.2450 - val_accuracy: 0.7383
Epoch 2/100
32/32 [==============================] - 0s 4ms/step - loss: 3.1204 - accuracy: 0.6641 - val_loss: 1.2224 - val_accuracy: 0.7656
Epoch 3/100
32/32 [==============================] - 0s 4ms/step - loss: 0.8795 - accuracy: 0.8145 - val_loss: 0.5029 - val_accuracy: 0.8789
Epoch 4/100
32/32 [==============================] - 0s 4ms/step - loss: 0.4347 - accuracy: 0.8779 - val_loss: 0.3321 - val_accuracy: 0.9102
Epoch 5/100
32/32 [==============================] - 0s 4ms/step - loss: 0.3321 - accuracy: 0.8984 - val_loss: 0.3086 - val_accuracy: 0.9180
Epoch 6/100
32/32 [==============================] - 0s 4ms/step - loss: 0.3014 - accuracy: 0.9004 - val_loss: 0.2916 - val_accuracy: 0.9180
Epoch 7/100
32/32 [==============================] - 0s 4ms/step - loss: 0.2964 - accuracy: 0.8994 - val_loss: 0.3049 - val_accuracy: 0.8984
Epoch 8/100

#### 90:10

In [ ]:
X_train_90_10_np = np.array(X_train_90_10)
X_test_90_10_np = np.array(X_test_90_10)

LE = LabelEncoder()
y_train_90_10_enc = LE.fit_transform(y_train_90_10)
y_test_90_10_enc = LE.fit_transform(y_test_90_10)

In [ ]:
ANN_90_10 = Sequential()

ANN_90_10.add(Dense(units = 16, activation = 'relu', input_shape = (df.shape[1]-1,)))
ANN_90_10.add(Dense(units = 16, activation = 'relu'))
ANN_90_10.add(Dense(units = 1, activation = 'sigmoid'))
ANN_90_10.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ['accuracy'])

ANN_90_10.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_49 (Dense)            (None, 16)                192       
                                                                 
 dense_50 (Dense)            (None, 16)                272       
                                                                 
 dense_51 (Dense)            (None, 1)                 17        
                                                                 
Total params: 481
Trainable params: 481
Non-trainable params: 0
_________________________________________________________________


In [ ]:
ANN_90_10.fit(X_train_90_10_np, y_train_90_10_enc, epochs = 100, validation_split = 0.2)

Epoch 1/100
36/36 [==============================] - 2s 16ms/step - loss: 3.5961 - accuracy: 0.6936 - val_loss: 0.5609 - val_accuracy: 0.8403
Epoch 2/100
36/36 [==============================] - 0s 5ms/step - loss: 0.4713 - accuracy: 0.8568 - val_loss: 0.2627 - val_accuracy: 0.8993
Epoch 3/100
36/36 [==============================] - 0s 5ms/step - loss: 0.2790 - accuracy: 0.8967 - val_loss: 0.2113 - val_accuracy: 0.9271
Epoch 4/100
36/36 [==============================] - 0s 6ms/step - loss: 0.2046 - accuracy: 0.9253 - val_loss: 0.1969 - val_accuracy: 0.9340
Epoch 5/100
36/36 [==============================] - 0s 4ms/step - loss: 0.1976 - accuracy: 0.9262 - val_loss: 0.1838 - val_accuracy: 0.9444
Epoch 6/100
36/36 [==============================] - 0s 4ms/step - loss: 0.1810 - accuracy: 0.9323 - val_loss: 0.1915 - val_accuracy: 0.9410
Epoch 7/100
36/36 [==============================] - 0s 5ms/step - loss: 0.1863 - accuracy: 0.9410 - val_loss: 0.1782 - val_accuracy: 0.9410
Epoch 8/100


### Testing and evaluation (SVM) models

#### 10:90

##### Kernel RBF

In [ ]:
y_pred_SVM_10_90_RBF = SVM_10_90_RBF.predict(X_test_10_90)
acc_score_SVM_10_90_RBF = accuracy_score(y_pred_SVM_10_90_RBF, y_test_10_90)
clf_report_SVM_10_90_RBF = classification_report(y_pred_SVM_10_90_RBF, y_test_10_90)

print('Accuracy Score :\n{}'.format(acc_score_SVM_10_90_RBF))
print('\n\n')
print('Classification Report :\n{}'.format(clf_report_SVM_10_90_RBF))

Accuracy Score :
0.8701388888888889



Classification Report :
              precision    recall  f1-score   support

       cacat       0.98      0.80      0.88       881
 tidak_cacat       0.76      0.98      0.85       559

    accuracy                           0.87      1440
   macro avg       0.87      0.89      0.87      1440
weighted avg       0.90      0.87      0.87      1440



##### Kernel Polynomial

In [ ]:
y_pred_SVM_10_90_poly = SVM_10_90_poly.predict(X_test_10_90)
acc_score_SVM_10_90_poly = accuracy_score(y_pred_SVM_10_90_poly, y_test_10_90)
clf_report_SVM_10_90_poly = classification_report(y_pred_SVM_10_90_poly, y_test_10_90)

print('Accuracy Score :\n{}'.format(acc_score_SVM_10_90_poly))
print('\n\n')
print('Classification Report :\n{}'.format(clf_report_SVM_10_90_poly))

Accuracy Score :
0.8715277777777778



Classification Report :
              precision    recall  f1-score   support

       cacat       0.98      0.80      0.88       879
 tidak_cacat       0.76      0.98      0.86       561

    accuracy                           0.87      1440
   macro avg       0.87      0.89      0.87      1440
weighted avg       0.90      0.87      0.87      1440



##### Kernel Sigmoid

In [ ]:
y_pred_SVM_10_90_sigmoid = SVM_10_90_sigmoid.predict(X_test_10_90)
acc_score_SVM_10_90_sigmoid = accuracy_score(y_pred_SVM_10_90_sigmoid, y_test_10_90)
clf_report_SVM_10_90_sigmoid = classification_report(y_pred_SVM_10_90_sigmoid, y_test_10_90)

print('Accuracy Score :\n{}'.format(acc_score_SVM_10_90_sigmoid))
print('\n\n')
print('Classification Report :\n{}'.format(clf_report_SVM_10_90_sigmoid))

Accuracy Score :
0.5159722222222223



Classification Report :
              precision    recall  f1-score   support

       cacat       0.49      0.52      0.50       683
 tidak_cacat       0.54      0.52      0.53       757

    accuracy                           0.52      1440
   macro avg       0.52      0.52      0.52      1440
weighted avg       0.52      0.52      0.52      1440



#### 20:80

##### Kernel RBF

In [ ]:
y_pred_SVM_20_80_RBF = SVM_20_80_RBF.predict(X_test_20_80)
acc_score_SVM_20_80_RBF = accuracy_score(y_pred_SVM_20_80_RBF, y_test_20_80)
clf_report_SVM_20_80_RBF = classification_report(y_pred_SVM_20_80_RBF, y_test_20_80)

print('Accuracy Score :\n{}'.format(acc_score_SVM_20_80_RBF))
print('\n\n')
print('Classification Report :\n{}'.format(clf_report_SVM_20_80_RBF))

Accuracy Score :
0.87265625



Classification Report :
              precision    recall  f1-score   support

       cacat       0.98      0.81      0.89       779
 tidak_cacat       0.76      0.98      0.86       501

    accuracy                           0.87      1280
   macro avg       0.87      0.89      0.87      1280
weighted avg       0.90      0.87      0.87      1280



##### Kernel Polynomial

In [ ]:
y_pred_SVM_20_80_poly = SVM_20_80_poly.predict(X_test_20_80)
acc_score_SVM_20_80_poly = accuracy_score(y_pred_SVM_20_80_poly, y_test_20_80)
clf_report_SVM_20_80_poly = classification_report(y_pred_SVM_20_80_poly, y_test_20_80)

print('Accuracy Score :\n{}'.format(acc_score_SVM_20_80_poly))
print('\n\n')
print('Classification Report :\n{}'.format(clf_report_SVM_20_80_poly))

Accuracy Score :
0.86171875



Classification Report :
              precision    recall  f1-score   support

       cacat       0.98      0.79      0.88       793
 tidak_cacat       0.74      0.98      0.84       487

    accuracy                           0.86      1280
   macro avg       0.86      0.88      0.86      1280
weighted avg       0.89      0.86      0.86      1280



##### Kernel Sigmoid

In [ ]:
y_pred_SVM_20_80_sigmoid = SVM_20_80_sigmoid.predict(X_test_20_80)
acc_score_SVM_20_80_sigmoid = accuracy_score(y_pred_SVM_20_80_sigmoid, y_test_20_80)
clf_report_SVM_20_80_sigmoid = classification_report(y_pred_SVM_20_80_sigmoid, y_test_20_80)

print('Accuracy Score :\n{}'.format(acc_score_SVM_20_80_sigmoid))
print('\n\n')
print('Classification Report :\n{}'.format(clf_report_SVM_20_80_sigmoid))

Accuracy Score :
0.51953125



Classification Report :
              precision    recall  f1-score   support

       cacat       0.50      0.52      0.51       613
 tidak_cacat       0.54      0.52      0.53       667

    accuracy                           0.52      1280
   macro avg       0.52      0.52      0.52      1280
weighted avg       0.52      0.52      0.52      1280



#### 30:70

##### Kernel RBF

In [ ]:
y_pred_SVM_30_70_RBF = SVM_30_70_RBF.predict(X_test_30_70)
acc_score_SVM_30_70_RBF = accuracy_score(y_pred_SVM_30_70_RBF, y_test_30_70)
clf_report_SVM_30_70_RBF = classification_report(y_pred_SVM_30_70_RBF, y_test_30_70)

print('Accuracy Score :\n{}'.format(acc_score_SVM_30_70_RBF))
print('\n\n')
print('Classification Report :\n{}'.format(clf_report_SVM_30_70_RBF))

Accuracy Score :
0.8758928571428571



Classification Report :
              precision    recall  f1-score   support

       cacat       0.98      0.81      0.89       679
 tidak_cacat       0.77      0.98      0.86       441

    accuracy                           0.88      1120
   macro avg       0.88      0.89      0.87      1120
weighted avg       0.90      0.88      0.88      1120



##### Kernel Polynomial

In [ ]:
y_pred_SVM_30_70_poly = SVM_30_70_poly.predict(X_test_30_70)
acc_score_SVM_30_70_poly = accuracy_score(y_pred_SVM_30_70_poly, y_test_30_70)
clf_report_SVM_30_70_poly = classification_report(y_pred_SVM_30_70_poly, y_test_30_70)

print('Accuracy Score :\n{}'.format(acc_score_SVM_30_70_poly))
print('\n\n')
print('Classification Report :\n{}'.format(clf_report_SVM_30_70_poly))

Accuracy Score :
0.8678571428571429



Classification Report :
              precision    recall  f1-score   support

       cacat       0.98      0.80      0.88       688
 tidak_cacat       0.75      0.98      0.85       432

    accuracy                           0.87      1120
   macro avg       0.87      0.89      0.87      1120
weighted avg       0.89      0.87      0.87      1120



##### Kernel Sigmoid

In [ ]:
y_pred_SVM_30_70_sigmoid = SVM_30_70_sigmoid.predict(X_test_30_70)
acc_score_SVM_30_70_sigmoid = accuracy_score(y_pred_SVM_30_70_sigmoid, y_test_30_70)
clf_report_SVM_30_70_sigmoid = classification_report(y_pred_SVM_30_70_sigmoid, y_test_30_70)

print('Accuracy Score :\n{}'.format(acc_score_SVM_30_70_sigmoid))
print('\n\n')
print('Classification Report :\n{}'.format(clf_report_SVM_30_70_sigmoid))

Accuracy Score :
0.48839285714285713



Classification Report :
              precision    recall  f1-score   support

       cacat       0.52      0.49      0.51       599
 tidak_cacat       0.45      0.49      0.47       521

    accuracy                           0.49      1120
   macro avg       0.49      0.49      0.49      1120
weighted avg       0.49      0.49      0.49      1120



#### 40:60

##### Kernel RBF

In [ ]:
y_pred_SVM_40_60_RBF = SVM_40_60_RBF.predict(X_test_40_60)
acc_score_SVM_40_60_RBF = accuracy_score(y_pred_SVM_40_60_RBF, y_test_40_60)
clf_report_SVM_40_60_RBF = classification_report(y_pred_SVM_40_60_RBF, y_test_40_60)

print('Accuracy Score :\n{}'.format(acc_score_SVM_40_60_RBF))
print('\n\n')
print('Classification Report :\n{}'.format(clf_report_SVM_40_60_RBF))

Accuracy Score :
0.8791666666666667



Classification Report :
              precision    recall  f1-score   support

       cacat       0.98      0.81      0.89       578
 tidak_cacat       0.78      0.98      0.87       382

    accuracy                           0.88       960
   macro avg       0.88      0.90      0.88       960
weighted avg       0.90      0.88      0.88       960



##### Kernel Polynomial

In [ ]:
y_pred_SVM_40_60_poly = SVM_40_60_poly.predict(X_test_40_60)
acc_score_SVM_40_60_poly = accuracy_score(y_pred_SVM_40_60_poly, y_test_40_60)
clf_report_SVM_40_60_poly = classification_report(y_pred_SVM_40_60_poly, y_test_40_60)

print('Accuracy Score :\n{}'.format(acc_score_SVM_40_60_poly))
print('\n\n')
print('Classification Report :\n{}'.format(clf_report_SVM_40_60_poly))

Accuracy Score :
0.8770833333333333



Classification Report :
              precision    recall  f1-score   support

       cacat       0.98      0.81      0.89       580
 tidak_cacat       0.77      0.98      0.86       380

    accuracy                           0.88       960
   macro avg       0.88      0.89      0.88       960
weighted avg       0.90      0.88      0.88       960



##### Kernel Sigmoid

In [ ]:
y_pred_SVM_40_60_sigmoid = SVM_40_60_sigmoid.predict(X_test_40_60)
acc_score_SVM_40_60_sigmoid = accuracy_score(y_pred_SVM_40_60_sigmoid, y_test_40_60)
clf_report_SVM_40_60_sigmoid = classification_report(y_pred_SVM_40_60_sigmoid, y_test_40_60)

print('Accuracy Score :\n{}'.format(acc_score_SVM_40_60_sigmoid))
print('\n\n')
print('Classification Report :\n{}'.format(clf_report_SVM_40_60_sigmoid))

Accuracy Score :
0.49895833333333334



Classification Report :
              precision    recall  f1-score   support

       cacat       0.51      0.50      0.51       495
 tidak_cacat       0.48      0.50      0.49       465

    accuracy                           0.50       960
   macro avg       0.50      0.50      0.50       960
weighted avg       0.50      0.50      0.50       960



#### 50:50

##### Kernel RBF

In [ ]:
y_pred_SVM_50_50_RBF = SVM_50_50_RBF.predict(X_test_50_50)
acc_score_SVM_50_50_RBF = accuracy_score(y_pred_SVM_50_50_RBF, y_test_50_50)
clf_report_SVM_50_50_RBF = classification_report(y_pred_SVM_50_50_RBF, y_test_50_50)

print('Accuracy Score :\n{}'.format(acc_score_SVM_50_50_RBF))
print('\n\n')
print('Classification Report :\n{}'.format(clf_report_SVM_50_50_RBF))

Accuracy Score :
0.8775



Classification Report :
              precision    recall  f1-score   support

       cacat       0.98      0.81      0.89       484
 tidak_cacat       0.77      0.98      0.86       316

    accuracy                           0.88       800
   macro avg       0.88      0.89      0.88       800
weighted avg       0.90      0.88      0.88       800



##### Kernel Polynomial

In [ ]:
y_pred_SVM_50_50_poly = SVM_50_50_poly.predict(X_test_50_50)
acc_score_SVM_50_50_poly = accuracy_score(y_pred_SVM_50_50_poly, y_test_50_50)
clf_report_SVM_50_50_poly = classification_report(y_pred_SVM_50_50_poly, y_test_50_50)

print('Accuracy Score :\n{}'.format(acc_score_SVM_50_50_poly))
print('\n\n')
print('Classification Report :\n{}'.format(clf_report_SVM_50_50_poly))

Accuracy Score :
0.87375



Classification Report :
              precision    recall  f1-score   support

       cacat       0.98      0.81      0.89       487
 tidak_cacat       0.77      0.98      0.86       313

    accuracy                           0.87       800
   macro avg       0.87      0.89      0.87       800
weighted avg       0.90      0.87      0.88       800



##### Kernel Sigmoid

In [ ]:
y_pred_SVM_50_50_sigmoid = SVM_50_50_sigmoid.predict(X_test_50_50)
acc_score_SVM_50_50_sigmoid = accuracy_score(y_pred_SVM_50_50_sigmoid, y_test_50_50)
clf_report_SVM_50_50_sigmoid = classification_report(y_pred_SVM_50_50_sigmoid, y_test_50_50)

print('Accuracy Score :\n{}'.format(acc_score_SVM_50_50_sigmoid))
print('\n\n')
print('Classification Report :\n{}'.format(clf_report_SVM_50_50_sigmoid))

Accuracy Score :
0.50125



Classification Report :
              precision    recall  f1-score   support

       cacat       0.51      0.50      0.50       403
 tidak_cacat       0.50      0.50      0.50       397

    accuracy                           0.50       800
   macro avg       0.50      0.50      0.50       800
weighted avg       0.50      0.50      0.50       800



#### 60:40

##### Kernel RBF

In [ ]:
SVM_60_40_RBF = SVC(kernel = 'rbf')
SVM_60_40_RBF.fit(X_train_60_40, y_train_60_40)

SVC()

##### Kernel Polynomial

In [ ]:
SVM_60_40_poly = SVC(kernel = 'poly')
SVM_60_40_poly.fit(X_train_60_40, y_train_60_40)

SVC(kernel='poly')

##### Kernel Sigmoid

In [ ]:
SVM_60_40_sigmoid = SVC(kernel = 'sigmoid')
SVM_60_40_sigmoid.fit(X_train_60_40, y_train_60_40)

SVC(kernel='sigmoid')

#### 70:30

##### Kernel RBF

In [ ]:
y_pred_SVM_70_30_RBF = SVM_70_30_RBF.predict(X_test_70_30)
acc_score_SVM_70_30_RBF = accuracy_score(y_pred_SVM_70_30_RBF, y_test_70_30)
clf_report_SVM_70_30_RBF = classification_report(y_pred_SVM_70_30_RBF, y_test_70_30)

print('Accuracy Score :\n{}'.format(acc_score_SVM_70_30_RBF))
print('\n\n')
print('Classification Report :\n{}'.format(clf_report_SVM_70_30_RBF))

Accuracy Score :
0.8833333333333333



Classification Report :
              precision    recall  f1-score   support

       cacat       0.97      0.83      0.89       282
 tidak_cacat       0.80      0.96      0.87       198

    accuracy                           0.88       480
   macro avg       0.88      0.90      0.88       480
weighted avg       0.90      0.88      0.88       480



##### Kernel Polynomial

In [ ]:
y_pred_SVM_70_30_poly = SVM_70_30_poly.predict(X_test_70_30)
acc_score_SVM_70_30_poly = accuracy_score(y_pred_SVM_70_30_poly, y_test_70_30)
clf_report_SVM_70_30_poly = classification_report(y_pred_SVM_70_30_poly, y_test_70_30)

print('Accuracy Score :\n{}'.format(acc_score_SVM_70_30_poly))
print('\n\n')
print('Classification Report :\n{}'.format(clf_report_SVM_70_30_poly))

Accuracy Score :
0.8770833333333333



Classification Report :
              precision    recall  f1-score   support

       cacat       0.97      0.82      0.89       287
 tidak_cacat       0.78      0.97      0.86       193

    accuracy                           0.88       480
   macro avg       0.88      0.89      0.88       480
weighted avg       0.90      0.88      0.88       480



##### Kernel Sigmoid

In [ ]:
y_pred_SVM_70_30_sigmoid = SVM_70_30_sigmoid.predict(X_test_70_30)
acc_score_SVM_70_30_sigmoid = accuracy_score(y_pred_SVM_70_30_sigmoid, y_test_70_30)
clf_report_SVM_70_30_sigmoid = classification_report(y_pred_SVM_70_30_sigmoid, y_test_70_30)

print('Accuracy Score :\n{}'.format(acc_score_SVM_70_30_sigmoid))
print('\n\n')
print('Classification Report :\n{}'.format(clf_report_SVM_70_30_sigmoid))

Accuracy Score :
0.48541666666666666



Classification Report :
              precision    recall  f1-score   support

       cacat       0.46      0.48      0.47       229
 tidak_cacat       0.51      0.49      0.50       251

    accuracy                           0.49       480
   macro avg       0.49      0.49      0.49       480
weighted avg       0.49      0.49      0.49       480



#### 80:20

##### Kernel RBF

In [ ]:
y_pred_SVM_80_20_RBF = SVM_80_20_RBF.predict(X_test_80_20)
acc_score_SVM_80_20_RBF = accuracy_score(y_pred_SVM_80_20_RBF, y_test_80_20)
clf_report_SVM_80_20_RBF = classification_report(y_pred_SVM_80_20_RBF, y_test_80_20)

print('Accuracy Score :\n{}'.format(acc_score_SVM_80_20_RBF))
print('\n\n')
print('Classification Report :\n{}'.format(clf_report_SVM_80_20_RBF))

Accuracy Score :
0.884375



Classification Report :
              precision    recall  f1-score   support

       cacat       0.99      0.82      0.90       193
 tidak_cacat       0.78      0.98      0.87       127

    accuracy                           0.88       320
   macro avg       0.88      0.90      0.88       320
weighted avg       0.91      0.88      0.89       320



##### Kernel Polynomial

In [ ]:
y_pred_SVM_80_20_poly = SVM_80_20_poly.predict(X_test_80_20)
acc_score_SVM_80_20_poly = accuracy_score(y_pred_SVM_80_20_poly, y_test_80_20)
clf_report_SVM_80_20_poly = classification_report(y_pred_SVM_80_20_poly, y_test_80_20)

print('Accuracy Score :\n{}'.format(acc_score_SVM_80_20_poly))
print('\n\n')
print('Classification Report :\n{}'.format(clf_report_SVM_80_20_poly))

Accuracy Score :
0.878125



Classification Report :
              precision    recall  f1-score   support

       cacat       0.99      0.81      0.89       197
 tidak_cacat       0.76      0.99      0.86       123

    accuracy                           0.88       320
   macro avg       0.88      0.90      0.88       320
weighted avg       0.90      0.88      0.88       320



##### Kernel Sigmoid

In [ ]:
y_pred_SVM_80_20_sigmoid = SVM_80_20_sigmoid.predict(X_test_80_20)
acc_score_SVM_80_20_sigmoid = accuracy_score(y_pred_SVM_80_20_sigmoid, y_test_80_20)
clf_report_SVM_80_20_sigmoid = classification_report(y_pred_SVM_80_20_sigmoid, y_test_80_20)

print('Accuracy Score :\n{}'.format(acc_score_SVM_80_20_sigmoid))
print('\n\n')
print('Classification Report :\n{}'.format(clf_report_SVM_80_20_sigmoid))

Accuracy Score :
0.5



Classification Report :
              precision    recall  f1-score   support

       cacat       0.49      0.50      0.50       158
 tidak_cacat       0.51      0.50      0.50       162

    accuracy                           0.50       320
   macro avg       0.50      0.50      0.50       320
weighted avg       0.50      0.50      0.50       320



#### 90:10

##### Kernel RBF

In [ ]:
y_pred_SVM_90_10_RBF = SVM_90_10_RBF.predict(X_test_90_10)
acc_score_SVM_90_10_RBF = accuracy_score(y_pred_SVM_90_10_RBF, y_test_90_10)
clf_report_SVM_90_10_RBF = classification_report(y_pred_SVM_90_10_RBF, y_test_90_10)

print('Accuracy Score :\n{}'.format(acc_score_SVM_90_10_RBF))
print('\n\n')
print('Classification Report :\n{}'.format(clf_report_SVM_90_10_RBF))

Accuracy Score :
0.8875



Classification Report :
              precision    recall  f1-score   support

       cacat       0.97      0.83      0.90        94
 tidak_cacat       0.80      0.97      0.88        66

    accuracy                           0.89       160
   macro avg       0.89      0.90      0.89       160
weighted avg       0.90      0.89      0.89       160



##### Kernel Polynomial

In [ ]:
y_pred_SVM_90_10_poly = SVM_90_10_poly.predict(X_test_90_10)
acc_score_SVM_90_10_poly = accuracy_score(y_pred_SVM_90_10_poly, y_test_90_10)
clf_report_SVM_90_10_poly = classification_report(y_pred_SVM_90_10_poly, y_test_90_10)

print('Accuracy Score :\n{}'.format(acc_score_SVM_90_10_poly))
print('\n\n')
print('Classification Report :\n{}'.format(clf_report_SVM_90_10_poly))

Accuracy Score :
0.88125



Classification Report :
              precision    recall  f1-score   support

       cacat       0.99      0.81      0.89        97
 tidak_cacat       0.78      0.98      0.87        63

    accuracy                           0.88       160
   macro avg       0.88      0.90      0.88       160
weighted avg       0.90      0.88      0.88       160



##### Kernel Sigmoid

In [ ]:
y_pred_SVM_90_10_sigmoid = SVM_90_10_sigmoid.predict(X_test_90_10)
acc_score_SVM_90_10_sigmoid = accuracy_score(y_pred_SVM_90_10_sigmoid, y_test_90_10)
clf_report_SVM_90_10_sigmoid = classification_report(y_pred_SVM_90_10_sigmoid, y_test_90_10)

print('Accuracy Score :\n{}'.format(acc_score_SVM_90_10_sigmoid))
print('\n\n')
print('Classification Report :\n{}'.format(clf_report_SVM_90_10_sigmoid))

Accuracy Score :
0.5



Classification Report :
              precision    recall  f1-score   support

       cacat       0.55      0.50      0.52        88
 tidak_cacat       0.45      0.50      0.47        72

    accuracy                           0.50       160
   macro avg       0.50      0.50      0.50       160
weighted avg       0.51      0.50      0.50       160



### Testing and evaluation (ANN) models

#### 10:90

In [ ]:
y_pred_ANN_10_90 = np.round(ANN_10_90.predict(X_test_10_90_np), 0)
acc_score_ANN_10_90 = accuracy_score(y_pred_ANN_10_90, y_test_10_90_enc)
clf_report_ANN_10_90 = classification_report(y_pred_ANN_10_90, y_test_10_90_enc)

print('Accuracy Score :\n{}'.format(acc_score_ANN_10_90))
print('\n\n')
print('Classification Report :\n{}'.format(clf_report_ANN_10_90))

Accuracy Score :
0.9284722222222223



Classification Report :
              precision    recall  f1-score   support

         0.0       0.95      0.91      0.93       747
         1.0       0.91      0.95      0.93       693

    accuracy                           0.93      1440
   macro avg       0.93      0.93      0.93      1440
weighted avg       0.93      0.93      0.93      1440



#### 20:80

In [ ]:
y_pred_ANN_20_80 = np.round(ANN_20_80.predict(X_test_20_80_np), 0)
acc_score_ANN_20_80 = accuracy_score(y_pred_ANN_20_80, y_test_20_80_enc)
clf_report_ANN_20_80 = classification_report(y_pred_ANN_20_80, y_test_20_80_enc)

print('Accuracy Score :\n{}'.format(acc_score_ANN_20_80))
print('\n\n')
print('Classification Report :\n{}'.format(clf_report_ANN_20_80))

Accuracy Score :
0.9140625



Classification Report :
              precision    recall  f1-score   support

         0.0       0.89      0.93      0.91       610
         1.0       0.94      0.90      0.92       670

    accuracy                           0.91      1280
   macro avg       0.91      0.91      0.91      1280
weighted avg       0.92      0.91      0.91      1280



#### 30:70

In [ ]:
y_pred_ANN_30_70 = np.round(ANN_30_70.predict(X_test_30_70_np), 0)
acc_score_ANN_30_70 = accuracy_score(y_pred_ANN_30_70, y_test_30_70_enc)
clf_report_ANN_30_70 = classification_report(y_pred_ANN_30_70, y_test_30_70_enc)

print('Accuracy Score :\n{}'.format(acc_score_ANN_30_70))
print('\n\n')
print('Classification Report :\n{}'.format(clf_report_ANN_30_70))

Accuracy Score :
0.9375



Classification Report :
              precision    recall  f1-score   support

         0.0       0.98      0.90      0.94       606
         1.0       0.90      0.98      0.93       514

    accuracy                           0.94      1120
   macro avg       0.94      0.94      0.94      1120
weighted avg       0.94      0.94      0.94      1120



#### 40:60

In [ ]:
y_pred_ANN_40_60 = np.round(ANN_40_60.predict(X_test_40_60_np), 0)
acc_score_ANN_40_60 = accuracy_score(y_pred_ANN_40_60, y_test_40_60_enc)
clf_report_ANN_40_60 = classification_report(y_pred_ANN_40_60, y_test_40_60_enc)

print('Accuracy Score :\n{}'.format(acc_score_ANN_40_60))
print('\n\n')
print('Classification Report :\n{}'.format(clf_report_ANN_40_60))

Accuracy Score :
0.9416666666666667



Classification Report :
              precision    recall  f1-score   support

         0.0       0.95      0.93      0.94       490
         1.0       0.93      0.95      0.94       470

    accuracy                           0.94       960
   macro avg       0.94      0.94      0.94       960
weighted avg       0.94      0.94      0.94       960



#### 50:50

In [ ]:
y_pred_ANN_50_50 = np.round(ANN_50_50.predict(X_test_50_50_np), 0)
acc_score_ANN_50_50 = accuracy_score(y_pred_ANN_50_50, y_test_50_50_enc)
clf_report_ANN_50_50 = classification_report(y_pred_ANN_50_50, y_test_50_50_enc)

print('Accuracy Score :\n{}'.format(acc_score_ANN_50_50))
print('\n\n')
print('Classification Report :\n{}'.format(clf_report_ANN_50_50))

Accuracy Score :
0.9475



Classification Report :
              precision    recall  f1-score   support

         0.0       0.97      0.93      0.95       418
         1.0       0.93      0.97      0.95       382

    accuracy                           0.95       800
   macro avg       0.95      0.95      0.95       800
weighted avg       0.95      0.95      0.95       800



#### 60:40

In [ ]:
y_pred_ANN_60_40 = np.round(ANN_60_40.predict(X_test_60_40_np), 0)
acc_score_ANN_60_40 = accuracy_score(y_pred_ANN_60_40, y_test_60_40_enc)
clf_report_ANN_60_40 = classification_report(y_pred_ANN_60_40, y_test_60_40_enc)

print('Accuracy Score :\n{}'.format(acc_score_ANN_60_40))
print('\n\n')
print('Classification Report :\n{}'.format(clf_report_ANN_60_40))

Accuracy Score :
0.940625



Classification Report :
              precision    recall  f1-score   support

         0.0       0.97      0.91      0.94       342
         1.0       0.91      0.97      0.94       298

    accuracy                           0.94       640
   macro avg       0.94      0.94      0.94       640
weighted avg       0.94      0.94      0.94       640



#### 70:30

In [ ]:
y_pred_ANN_70_30 = np.round(ANN_70_30.predict(X_test_70_30_np), 0)
acc_score_ANN_70_30 = accuracy_score(y_pred_ANN_70_30, y_test_70_30_enc)
clf_report_ANN_70_30 = classification_report(y_pred_ANN_70_30, y_test_70_30_enc)

print('Accuracy Score :\n{}'.format(acc_score_ANN_70_30))
print('\n\n')
print('Classification Report :\n{}'.format(clf_report_ANN_70_30))

Accuracy Score :
0.95625



Classification Report :
              precision    recall  f1-score   support

         0.0       0.98      0.94      0.96       251
         1.0       0.93      0.98      0.96       229

    accuracy                           0.96       480
   macro avg       0.96      0.96      0.96       480
weighted avg       0.96      0.96      0.96       480



#### 80:20

In [ ]:
y_pred_ANN_80_20 = np.round(ANN_80_20.predict(X_test_80_20_np), 0)
acc_score_ANN_80_20 = accuracy_score(y_pred_ANN_80_20, y_test_80_20_enc)
clf_report_ANN_80_20 = classification_report(y_pred_ANN_80_20, y_test_80_20_enc)

print('Accuracy Score :\n{}'.format(acc_score_ANN_80_20))
print('\n\n')
print('Classification Report :\n{}'.format(clf_report_ANN_80_20))

Accuracy Score :
0.9375



Classification Report :
              precision    recall  f1-score   support

         0.0       0.95      0.93      0.94       164
         1.0       0.93      0.95      0.94       156

    accuracy                           0.94       320
   macro avg       0.94      0.94      0.94       320
weighted avg       0.94      0.94      0.94       320



#### 90:10

In [ ]:
y_pred_ANN_90_10 = np.round(ANN_90_10.predict(X_test_90_10_np), 0)
acc_score_ANN_90_10 = accuracy_score(y_pred_ANN_90_10, y_test_90_10_enc)
clf_report_ANN_90_10 = classification_report(y_pred_ANN_90_10, y_test_90_10_enc)

print('Accuracy Score :\n{}'.format(acc_score_ANN_90_10))
print('\n\n')
print('Classification Report :\n{}'.format(clf_report_ANN_90_10))

Accuracy Score :
0.925



Classification Report :
              precision    recall  f1-score   support

         0.0       0.94      0.91      0.93        82
         1.0       0.91      0.94      0.92        78

    accuracy                           0.93       160
   macro avg       0.93      0.93      0.92       160
weighted avg       0.93      0.93      0.93       160

